<a href="https://www.kaggle.com/code/chingiztursunbayev/airline-customer-satisfaction?scriptVersionId=176585421" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

In [ ]:
train_data = pd.read_csv('/kaggle/input/aviakompaniya/train_dataset.csv')
test_data = pd.read_csv('/kaggle/input/aviakompaniya/test_dataset.csv')

In [ ]:
train_data.shape, test_data.shape

In [ ]:
test_data.isna().sum()

In [ ]:
train_data.isna().sum()

In [ ]:
train_data['Arrival Delay in Minutes'] = train_data['Arrival Delay in Minutes'].fillna(28)

test_data['Arrival Delay in Minutes'] = test_data['Arrival Delay in Minutes'].fillna(19)

In [ ]:
train_data = pd.get_dummies(train_data, columns=['Gender', 'Customer Type', 'Type of Travel', 'Class'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['Gender', 'Customer Type', 'Type of Travel', 'Class'], drop_first=True)

In [ ]:
train_data.head(10)

In [ ]:
X_train = train_data.drop(columns=['id', 'satisfaction'])
y_train = train_data['satisfaction']
X_test = test_data.drop(columns=['id'])

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled

In [ ]:
model = RandomForestClassifier()
model.fit(X_train_scaled, y_train)

In [ ]:
predictions = model.predict(X_test_scaled)
predictions

In [ ]:
type(predictions)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score


cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')

mean_cv_score = cv_scores.mean()
std_cv_score = cv_scores.std()

print("Mean Cross-Validation Accuracy:", mean_cv_score)
print("Standard Deviation of Cross-Validation Accuracy:", std_cv_score)

In [ ]:
results = pd.DataFrame({'id': test_data['id'], 'satisfaction': predictions})
results.to_csv('predictions.csv', index=False)

In [ ]:
results